In [1]:
%%time
!pip install --target=/kaggle/working --extra-index-url=https://pypi.nvidia.com "cudf-cu12==25.2.*" "cuml-cu12==25.2.*"
!rm -rf /kaggle/working/numpy*

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 64.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.7/557.7 MB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 522.8/522.8 kB 180.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 69.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.3/27.3 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 106.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 125.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 72.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.

In [2]:
!pip install scikit-learn==1.5.2 koolbox


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 6.2 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.


In [3]:
import cudf
cudf.__version__

'25.02.02'

In [4]:

import cuml
cuml

<module 'cuml' from '/kaggle/working/cuml/__init__.py'>

In [5]:
%load_ext cudf.pandas

import numpy as np, pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 500)

VER=1

In [6]:
# from sklearn.feature_selection import mutual_info_regression
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from scipy.special import logit
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
import joblib
import shutil
import optuna
import glob
import json

warnings.filterwarnings('ignore')

In [7]:
train=pd.read_csv('/kaggle/input/playground-series-s5e8/train.csv',index_col='id')
test=pd.read_csv('/kaggle/input/playground-series-s5e8/test.csv',index_col='id')
train2=pd.read_csv('/kaggle/input/bank-marketing-dataset-full/bank-full.csv',delimiter=";")
augment_data = pd.read_csv('/kaggle/input/data-augmentation/df_synth.csv')

train.shape

(750000, 17)

In [8]:
train.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
id,,,,,,,,,,,,,,,,,
0,42,technician,married,secondary,no,7,no,no,cellular,25,aug,117,3,-1,0,unknown,0
1,38,blue-collar,married,secondary,no,514,no,no,unknown,18,jun,185,1,-1,0,unknown,0
2,36,blue-collar,married,secondary,no,602,yes,no,unknown,14,may,111,2,-1,0,unknown,0
3,27,student,single,secondary,no,34,yes,no,unknown,28,may,10,2,-1,0,unknown,0
4,26,technician,married,secondary,no,889,yes,no,cellular,3,feb,902,1,-1,0,unknown,1


In [9]:
train2.y=train2.y.map({"no":0,'yes':1})

In [10]:
augment_data.columns=train.columns

In [11]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df=pd.DataFrame()
dict_lab=dict()
for col in train2.select_dtypes(include='object').columns:
    df[col] = le.fit_transform(train2[col])
    label_dict = {label: idx for idx, label in enumerate(le.classes_)}
    dict_lab[col]=label_dict

for key in dict_lab.keys():
    augment_data[key]=augment_data[key].map({k:v for v,k in dict_lab[key].items()})


In [12]:
train3=augment_data[augment_data.y==1]
train = pd.concat([train,train2,train3],axis=0,ignore_index=True).sample(frac=1.0).reset_index(drop=True)
print("Combined Train shape", train.shape)

Combined Train shape (862628, 17)


In [13]:
train.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,42,blue-collar,single,unknown,no,484,yes,no,unknown,15,may,647,12,-1,0,unknown,0
1,36,blue-collar,married,secondary,no,1364,no,no,unknown,11,jun,114,2,-1,0,unknown,0
2,36,blue-collar,married,secondary,no,707,yes,no,cellular,6,may,113,2,-1,0,unknown,0
3,52,technician,married,secondary,no,1663,no,no,telephone,3,feb,15,1,-1,0,unknown,0
4,45,services,married,secondary,no,-158,yes,no,cellular,7,jul,129,1,-1,0,unknown,0


In [14]:
CATS =[t for t in train.select_dtypes(exclude=np.number).columns ]
print(f"There are {len(CATS)} categorical columns:")
print( CATS )
print(f"There are 1 numerical column:")
print( ["balance"] )

There are 9 categorical columns:
['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']
There are 1 numerical column:
['balance']


In [15]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold,KFold
import numpy as np


In [16]:
def create_advanced_features(df):
    df = df.copy()
    
    # Basic binary features
    df['balance_positive'] = (df['balance'] > 0).astype(int)
    df['has_previous'] = (df['previous'] > 0).astype(int)
    df['duration_long'] = (df['duration'] > 300).astype(int)
    df['duration_very_long'] = (df['duration'] > 600).astype(int)
    df['campaign_multiple'] = (df['campaign'] > 2).astype(int)
    df['campaign_high'] = (df['campaign'] > 5).astype(int)
    
    # Enhanced numerical transformations
    df['log_duration'] = np.log1p(df['duration'])
    df['sqrt_age'] = np.sqrt(df['age'])
    df['log_balance'] = np.sign(df['balance']) * np.log1p(np.abs(df['balance']))
    df['age_squared'] = df['age'] ** 2
    df['duration_squared'] = df['duration'] ** 2
    df['campaign_squared'] = df['campaign'] ** 2
    
    # Detailed age binning
    df['age_bin'] = pd.cut(df['age'], bins=[0, 25, 30, 35, 40, 45, 50, 55, 60, 65, 100], labels=False)
    
    # Duration binning
    df['duration_bin'] = pd.qcut(df['duration'], q=15, labels=False, duplicates='drop')
    
    # Balance binning
    df['balance_bin'] = pd.qcut(df['balance'], q=10, labels=False, duplicates='drop')
    
    # Feature interactions
    df['age_duration'] = df['age'] * df['duration']
    df['age_campaign'] = df['age'] * df['campaign']
    df['balance_duration'] = df['balance'] * df['duration']
    df['campaign_duration'] = df['campaign'] * df['duration']
    df['age_balance'] = df['age'] * df['balance']
    df['previous_campaign'] = df['previous'] * df['campaign']
    
    # Ratios and rates
    df['duration_per_campaign'] = df['duration'] / (df['campaign'] + 1)
    df['balance_per_age'] = df['balance'] / (df['age'] + 1)
    df['duration_per_age'] = df['duration'] / (df['age'] + 1)
    df['campaign_per_age'] = df['campaign'] / (df['age'] + 1)
    
    # Contact-related features
    df['contact_success_rate'] = df['previous'] / (df['campaign'] + 1)
    df['pdays_binned'] = pd.cut(df['pdays'], bins=[-2, -1, 0, 50, 100, 200, 300, 1000], labels=False)
    df['has_pdays'] = (df['pdays'] != -1).astype(int)
    
    # Seasonal features
    month_map = {'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6,
                 'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12}
    df['month_num'] = df['month'].map(month_map)
    df['quarter'] = ((df['month_num'] - 1) // 3) + 1
    df['is_summer'] = df['month'].isin(['jun', 'jul', 'aug']).astype(int)
    df['is_winter'] = df['month'].isin(['dec', 'jan', 'feb']).astype(int)

    
    return df

train =   create_advanced_features(train)
test =   create_advanced_features(test)
# test_clean =   create_advanced_features(test_clean)


In [17]:
train

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,balance_positive,has_previous,duration_long,duration_very_long,campaign_multiple,campaign_high,log_duration,sqrt_age,log_balance,age_squared,duration_squared,campaign_squared,age_bin,duration_bin,balance_bin,age_duration,age_campaign,balance_duration,campaign_duration,age_balance,previous_campaign,duration_per_campaign,balance_per_age,duration_per_age,campaign_per_age,contact_success_rate,pdays_binned,has_pdays,month_num,quarter,is_summer,is_winter
0,42,blue-collar,single,unknown,no,484,yes,no,unknown,15,may,647,12,-1,0,unknown,0,1,0,1,1,1,1,6.473891,6.480741,6.184149,1764,418609,144,4,13,4,27174,504,313148,7764,20328,0,49.769231,11.255814,15.046512,0.279070,0.0,0,0,5,2,0,0
1,36,blue-collar,married,secondary,no,1364,no,no,unknown,11,jun,114,2,-1,0,unknown,0,1,0,0,0,0,0,4.744932,6.000000,7.218910,1296,12996,4,3,5,7,4104,72,155496,228,49104,0,38.000000,36.864865,3.081081,0.054054,0.0,0,0,6,2,1,0
2,36,blue-collar,married,secondary,no,707,yes,no,cellular,6,may,113,2,-1,0,unknown,0,1,0,0,0,0,0,4.736198,6.000000,6.562444,1296,12769,4,3,5,5,4068,72,79891,226,25452,0,37.666667,19.108108,3.054054,0.054054,0.0,0,0,5,2,0,0
3,52,technician,married,secondary,no,1663,no,no,telephone,3,feb,15,1,-1,0,unknown,0,1,0,0,0,0,0,2.772589,7.211103,7.416980,2704,225,1,6,0,7,780,52,24945,15,86476,0,7.500000,31.377358,0.283019,0.018868,0.0,0,0,2,1,0,1
4,45,services,married,secondary,no,-158,yes,no,cellular,7,jul,129,1,-1,0,unknown,0,0,0,0,0,0,0,4.867534,6.708204,-5.068904,2025,16641,1,4,6,0,5805,45,-20382,129,-7110,0,64.500000,-3.434783,2.804348,0.021739,0.0,0,0,7,3,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
862623,45,management,divorced,secondary,no,565,no,no,cellular,8,aug,90,2,-1,0,unknown,0,1,0,0,0,0,0,4.510860,6.708204,6.338594,2025,8100,4,4,3,4,4050,90,50850,180,25425,0,30.000000,12.282609,1.956522,0.043478,0.0,0,0,8,3,1,0
862624,33,technician,divorced,secondary,no,0,no,no,cellular,21,aug,92,2,-1,0,unknown,0,0,0,0,0,0,0,4.532599,5.744563,0.000000,1089,8464,4,2,3,1,3036,66,0,184,0,0,30.666667,0.000000,2.705882,0.058824,0.0,0,0,8,3,1,0
862625,57,retired,married,secondary,no,8648,no,no,cellular,17,nov,528,1,-1,0,unknown,0,1,0,1,0,0,0,6.270988,7.549834,9.065199,3249,278784,1,7,12,9,30096,57,4566144,528,492936,0,264.000000,149.103448,9.103448,0.017241,0.0,0,0,11,4,0,0
862626,36,admin.,married,secondary,no,-267,yes,no,cellular,7,jul,159,2,-1,0,unknown,0,0,0,0,0,0,0,5.075174,6.000000,-5.590987,1296,25281,4,3,8,0,5724,72,-42453,318,-9612,0,53.000000,-7.216216,4.297297,0.054054,0.0,0,0,7,3,1,0


In [18]:
Num=[]
for i in train.select_dtypes(np.number).columns.tolist():
    print(i,train[i].nunique())
    if i !="y" and i !="balance":
        Num.append(i)

age 78
balance 11431
day 31
duration 1908
campaign 52
pdays 662
previous 51
y 2
balance_positive 2
has_previous 2
duration_long 2
duration_very_long 2
campaign_multiple 2
campaign_high 2
log_duration 1908
sqrt_age 78
log_balance 11431
age_squared 78
duration_squared 1908
campaign_squared 52
age_bin 10
duration_bin 15
balance_bin 10
age_duration 29647
age_campaign 934
balance_duration 296636
campaign_duration 7487
age_balance 102061
previous_campaign 168
duration_per_campaign 11540
balance_per_age 122737
duration_per_age 40924
campaign_per_age 1280
contact_success_rate 277
pdays_binned 7
has_pdays 2
month_num 12
quarter 4
is_summer 2
is_winter 2


In [19]:
COMBO = []
for i,c in enumerate(CATS):
    #print(f"{c}, ",end="")
    combine = pd.concat([train[c],test[c]],axis=0)
    combine,_ = pd.factorize(combine)
    train[c] = combine[:len(train)]
    test[c] = combine[len(train):]
    n = f"{c}_wc"
    train[n] = train[c]*100 + train["balance"]
    test[n] = test[c]*100 + test["balance"]
    COMBO.append(n)
print()
print(f"We engineer {len(COMBO)} new columns!")
print( COMBO )


We engineer 9 new columns!
['job_wc', 'marital_wc', 'education_wc', 'default_wc', 'housing_wc', 'loan_wc', 'contact_wc', 'month_wc', 'poutcome_wc']


In [20]:
FEATURES = CATS + ["balance"] + COMBO + Num
print(f"We now have {len(FEATURES)} columns:")
print( FEATURES )

We now have 57 columns:
['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome', 'balance', 'job_wc', 'marital_wc', 'education_wc', 'default_wc', 'housing_wc', 'loan_wc', 'contact_wc', 'month_wc', 'poutcome_wc', 'age', 'day', 'duration', 'campaign', 'pdays', 'previous', 'balance_positive', 'has_previous', 'duration_long', 'duration_very_long', 'campaign_multiple', 'campaign_high', 'log_duration', 'sqrt_age', 'log_balance', 'age_squared', 'duration_squared', 'campaign_squared', 'age_bin', 'duration_bin', 'balance_bin', 'age_duration', 'age_campaign', 'balance_duration', 'campaign_duration', 'age_balance', 'previous_campaign', 'duration_per_campaign', 'balance_per_age', 'duration_per_age', 'campaign_per_age', 'contact_success_rate', 'pdays_binned', 'has_pdays', 'month_num', 'quarter', 'is_summer', 'is_winter']


In [21]:
# STATISTICS TO AGGEGATE FOR OUR FEATURE GROUPS
STATS = ["mean","std","count","nunique","median","min","max","skew"]
STATS2 = ["mean","std"]

In [22]:
scores = {}
oof_pred_probs = {}
test_pred_probs = {}

In [23]:
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import HistGradientBoostingClassifier

import keras
from keras.models import Sequential
from keras import layers
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers


2025-08-09 17:35:53.620234: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754760953.800847      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754760953.854366      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [24]:
def build_model(cat_features, num_features):
    
    x_input_cats = layers.Input(shape=(len(cat_features),))
    embs = []
    for j in range(len(cat_features)):
        e = layers.Embedding(cat_features_card[j], int(np.ceil(np.sqrt(cat_features_card[j]))))
        x = e(x_input_cats[:,j])
        x = layers.Flatten()(x)
        embs.append(x)
        
    x_input_nums = layers.Input(shape=(len(num_features),))
    
    x = layers.Concatenate(axis=-1)(embs+[x_input_nums]) 
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(.3)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(.3)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(.3)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(1, activation='sigmoid')(x)

    model = keras.Model(inputs=[x_input_cats,x_input_nums], outputs=x)
    return model

In [25]:
cat_features_card = []
for f in CATS:
        cat_features_card.append(train[f].nunique())
    
 

In [26]:
oof_pred_probs = {}
test_pred_probs = {}

In [27]:
%%time
FOLDS = 7
kf = KFold(n_splits=FOLDS, shuffle=True, random_state=42)
def train_model ( mode='xgb'):
    oof = np.zeros((len(train)))
    pred = np.zeros((len(test)))
    
    # OUTER K FOLD
    for i, (train_index, test_index) in enumerate(kf.split(train)):
        print(f"### OUTER Fold {i+1} ###")
    
        X_train = train.loc[train_index,FEATURES+['y']].reset_index(drop=True).copy()
        y_train = train.loc[train_index,'y']
    
        X_valid = train.loc[test_index,FEATURES].reset_index(drop=True).copy()
        y_valid = train.loc[test_index,'y']
    
        X_test = test[FEATURES].reset_index(drop=True).copy()
    
        # INNER K FOLD (TO PREVENT LEAKAGE WHEN USING PRICE)
        kf2 = KFold(n_splits=FOLDS, shuffle=True, random_state=42)   
        for j, (train_index2, test_index2) in enumerate(kf2.split(X_train)):
            print(f" ## INNER Fold {j+1} (outer fold {i+1}) ##")
    
            X_train2 = X_train.loc[train_index2,FEATURES+['y']].copy()
            X_valid2 = X_train.loc[test_index2,FEATURES].copy()
    
            ### FEATURE SET 1 (uses price) ###
            col = "balance"
            tmp = X_train2.groupby(col).y.agg(STATS)
            tmp.columns = [f"TE1_wc_{s}" for s in STATS]
            X_valid2 = X_valid2.merge(tmp, on=col, how="left")
            for c in tmp.columns:
                X_train.loc[test_index2,c] = X_valid2[c].values
    
            ### FEATURE SET 2 (uses price) ###
            for col in COMBO:
                tmp = X_train2.groupby(col).y.agg(STATS2)
                tmp.columns = [f"TE2_{col}_{s}" for s in STATS2]
                X_valid2 = X_valid2.merge(tmp, on=col, how="left")
                for c in tmp.columns:
                    X_train.loc[test_index2,c] = X_valid2[c].values
    
        ### FEATURE SET 1 (uses price) ###
        col = "balance"
        tmp = X_train.groupby(col).y.agg(STATS)
        tmp.columns = [f"TE1_wc_{s}" for s in STATS]
        X_valid = X_valid.merge(tmp, on=col, how="left")
        X_test = X_test.merge(tmp, on=col, how="left")
    
        ### FEATURE SET 2 (uses price) ###
        for col in COMBO:
            tmp = X_train.groupby(col).y.agg(STATS2)
            tmp.columns = [f"TE2_{col}_{s}" for s in STATS2]
            X_valid = X_valid.merge(tmp, on=col, how="left")
            X_test = X_test.merge(tmp, on=col, how="left")
    
        ### FEATURE SET 3 (does not use price) ###
        for col in CATS:
            col2 = "balance"
            tmp = X_train.groupby(col)[col2].agg(STATS2)
            tmp.columns = [f"FE3_{col}_wc_{s}" for s in STATS2]
            X_train = X_train.merge(tmp, on=col, how="left")
            X_valid = X_valid.merge(tmp, on=col, how="left")
            X_test = X_test.merge(tmp, on=col, how="left")
    
        # CONVERT TO CATS SO XGBOOST RECOGNIZES THEM
        if mode =='NN':
            for col in X_train.drop('y',axis=1).columns.tolist():
                X_train[col]=X_train[col].fillna(X_train[col].mean())
                X_valid[col]=X_valid[col].fillna(X_valid[col].mean())
                X_test[col]=X_test[col].fillna(X_test[col].mean())
        else:
            X_train[CATS] = X_train[CATS].astype("category")
            X_valid[CATS] = X_valid[CATS].astype("category")
            X_test[CATS] = X_test[CATS].astype("category")
     
 
        # DROP PRICE THAT WAS USED FOR TARGET ENCODING
        X_train = X_train.drop(['y'],axis=1)
    
         
        # Build model
        if mode ==  'xgb':
            
            model = XGBClassifier(
                device="cuda",  # uses GPU
                max_depth=6,
                colsample_bytree=0.5,
                subsample=0.8,
                n_estimators=10_000,
                learning_rate=0.02,
                enable_categorical=True,
                min_child_weight=10,
                eval_metric="auc",  # use AUC as evaluation metric
                use_label_encoder=False,
            )
        elif mode ==  'cat':
            model = CatBoostClassifier(**{'random_state': 42,
                                 'cat_features': CATS,
                                  'eval_metric': "Logloss",
                                 'n_estimators' : 5000,
                                 'learning_rate': 0.06524873965257823,
                                 'l2_leaf_reg': 0.8867612905712001,
                                 'bagging_temperature': 0.1317347791955057,
                                 'random_strength': 0.9922857768340815,
                                 'depth': 7,
                                 'min_data_in_leaf': 8,
                                 'task_type': "GPU"
                                 })
        elif mode ==  'lgbm':
              model = LGBMClassifier(**{'random_state': 42,
                              'early_stopping_round': 100,
                              'verbose': -1,
                              'n_estimators': 10000,
                              'metric': 'AUC',
                              'objective': 'binary',
                              'max_depth': 16,
                              'learning_rate': 0.007366917567300051,
                              'min_child_samples': 164,
                              'subsample': 0.9022880020285295,
                              'colsample_bytree': 0.4213201532077694,
                              'num_leaves': 122, 
                              'reg_alpha': 1.083996192298843,
                              'reg_lambda': 0.0700057221912873
                              })
        elif mode ==  'rfc':
 
            model = HistGradientBoostingClassifier(
                learning_rate=0.02,
                max_iter=10_000,
                max_depth=6,
                max_leaf_nodes=31,
                min_samples_leaf=10,
                l2_regularization=1.0,
                early_stopping=True,
                scoring='roc_auc',  # used only when early_stopping=True
                validation_fraction=0.1,
                n_iter_no_change=100,
                categorical_features=CATS,  # only needed if data is NumPy array
                random_state=42,
                verbose=0
            )
        elif mode == 'NN':
                num_features = [f for f in X_train.columns.tolist() if f not in CATS]
            
                X_train_cats = X_train[CATS]
                X_train_nums = X_train[num_features]

                X_val_cats = X_valid[CATS]
                X_val_nums = X_valid[num_features]

                X_test_cats = X_test[CATS]
                X_test_nums = X_test[num_features]

                model = build_model(CATS, num_features)                        
                keras.utils.set_random_seed(42)
                optimizer = keras.optimizers.AdamW(learning_rate=1e-2, weight_decay=1e-3)
                model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['auc'])
                 
               
        # TRAIN MODEL
        COLS = X_train.columns
        if mode =='lgbm':
            model.fit(
            X_train[COLS], y_train,
            eval_set=[(X_valid[COLS], y_valid)],
        )
        elif mode =='rfc':
       
            model.fit(
                X_train[COLS], y_train,
            )
        elif mode =='NN':
             
             model.fit([X_train_cats,X_train_nums], y_train, 
                          validation_data=([X_val_cats, X_val_nums], y_valid),
                          epochs=20,
                          batch_size=1000,
                          callbacks=[keras.callbacks.ReduceLROnPlateau(patience=1),
                                     keras.callbacks.EarlyStopping(patience=3)
                                    ])
        else:
            model.fit(
                X_train[COLS], y_train,
                eval_set=[(X_valid[COLS], y_valid)],  
                verbose=300,
            )
        if mode =="NN":
            oof[test_index] = model.predict([X_val_cats, X_val_nums]).squeeze()                      
            pred += model.predict([X_test_cats, X_test_nums]).squeeze() 
            
        else:
            oof[test_index] = model.predict_proba(X_valid[COLS])[:,1]
            pred += model.predict_proba(X_test[COLS])[:,1]
        
    pred /= FOLDS
    oof_pred_probs[mode]=oof
    test_pred_probs[mode]=pred
    return oof_pred_probs ,test_pred_probs 

CPU times: user 27 µs, sys: 0 ns, total: 27 µs
Wall time: 32.4 µs


In [28]:
# COMPUTE OVERALL CV SCORE
true = train.y.values
for m in ["xgb",'lgbm','cat','rfc',"NN"]:
    
    oof_pred_probs ,test_pred_probs =train_model(mode=m)
    print(f"{m} , done")
    s=roc_auc_score(true,oof_pred_probs[m])
    print(f"=> {m} Overall CV Score = {s}")

# => xgb Overall CV Score = 0.9675705159714821 BASELINE
# => xgb Overall CV Score   0.9680042763069224 FE
# => xgb Overall CV Score = 0.9661948377036753
# => cat Overall CV Score = 0.9669386996298588
# => lgbm Overall CV Score = 0.9686827545526355


### OUTER Fold 1 ###
 ## INNER Fold 1 (outer fold 1) ##
 ## INNER Fold 2 (outer fold 1) ##
 ## INNER Fold 3 (outer fold 1) ##
 ## INNER Fold 4 (outer fold 1) ##
 ## INNER Fold 5 (outer fold 1) ##
 ## INNER Fold 6 (outer fold 1) ##
 ## INNER Fold 7 (outer fold 1) ##
[0]	validation_0-auc:0.95444
[300]	validation_0-auc:0.97342
[600]	validation_0-auc:0.97621
[900]	validation_0-auc:0.97727
[1200]	validation_0-auc:0.97783
[1500]	validation_0-auc:0.97818
[1800]	validation_0-auc:0.97842
[2100]	validation_0-auc:0.97859
[2400]	validation_0-auc:0.97873
[2700]	validation_0-auc:0.97883
[3000]	validation_0-auc:0.97893
[3300]	validation_0-auc:0.97901
[3600]	validation_0-auc:0.97907
[3900]	validation_0-auc:0.97911
[4200]	validation_0-auc:0.97916
[4500]	validation_0-auc:0.97921
[4800]	validation_0-auc:0.97924
[5100]	validation_0-auc:0.97928
[5400]	validation_0-auc:0.97931
[5700]	validation_0-auc:0.97932
[6000]	validation_0-auc:0.97933
[6300]	validation_0-auc:0.97934
[6600]	validation_0-auc:0.97935
[690

0:	learn: 0.5885212	test: 0.5886233	best: 0.5886233 (0)	total: 188ms	remaining: 15m 40s
300:	learn: 0.1514796	test: 0.1524911	best: 0.1524911 (300)	total: 18.7s	remaining: 4m 51s
600:	learn: 0.1444597	test: 0.1483915	best: 0.1483915 (600)	total: 36.9s	remaining: 4m 29s
900:	learn: 0.1396644	test: 0.1467661	best: 0.1467661 (900)	total: 55.2s	remaining: 4m 11s
1200:	learn: 0.1356103	test: 0.1457869	best: 0.1457869 (1200)	total: 1m 13s	remaining: 3m 53s
1500:	learn: 0.1320942	test: 0.1451516	best: 0.1451480 (1497)	total: 1m 32s	remaining: 3m 35s
1800:	learn: 0.1287825	test: 0.1447906	best: 0.1447887 (1796)	total: 1m 50s	remaining: 3m 16s
2100:	learn: 0.1257796	test: 0.1444022	best: 0.1443865 (2087)	total: 2m 9s	remaining: 2m 58s
2400:	learn: 0.1230282	test: 0.1442133	best: 0.1442131 (2398)	total: 2m 28s	remaining: 2m 40s
2700:	learn: 0.1203336	test: 0.1440202	best: 0.1440114 (2691)	total: 2m 46s	remaining: 2m 22s
3000:	learn: 0.1178525	test: 0.1440058	best: 0.1439756 (2795)	total: 3m 5s	r

0:	learn: 0.5885746	test: 0.5887109	best: 0.5887109 (0)	total: 83.5ms	remaining: 6m 57s
300:	learn: 0.1506729	test: 0.1550457	best: 0.1550457 (300)	total: 18.6s	remaining: 4m 50s
600:	learn: 0.1437396	test: 0.1513192	best: 0.1513163 (599)	total: 37s	remaining: 4m 30s
900:	learn: 0.1389160	test: 0.1498130	best: 0.1498130 (900)	total: 55.1s	remaining: 4m 10s
1200:	learn: 0.1349759	test: 0.1489584	best: 0.1489569 (1198)	total: 1m 13s	remaining: 3m 52s
1500:	learn: 0.1314494	test: 0.1483549	best: 0.1483549 (1500)	total: 1m 31s	remaining: 3m 34s
1800:	learn: 0.1281832	test: 0.1480421	best: 0.1480421 (1800)	total: 1m 50s	remaining: 3m 16s
2100:	learn: 0.1251972	test: 0.1478179	best: 0.1478105 (2069)	total: 2m 9s	remaining: 2m 58s
2400:	learn: 0.1223982	test: 0.1476040	best: 0.1476040 (2400)	total: 2m 28s	remaining: 2m 40s
2700:	learn: 0.1197694	test: 0.1474096	best: 0.1473909 (2659)	total: 2m 46s	remaining: 2m 22s
3000:	learn: 0.1172079	test: 0.1473178	best: 0.1473108 (2879)	total: 3m 5s	rem

0:	learn: 0.5850485	test: 0.5851001	best: 0.5851001 (0)	total: 82.7ms	remaining: 6m 53s
300:	learn: 0.1510706	test: 0.1546918	best: 0.1546918 (300)	total: 18.7s	remaining: 4m 51s
600:	learn: 0.1439333	test: 0.1504772	best: 0.1504772 (600)	total: 37s	remaining: 4m 30s
900:	learn: 0.1392264	test: 0.1487211	best: 0.1487211 (900)	total: 55.3s	remaining: 4m 11s
1200:	learn: 0.1353399	test: 0.1477684	best: 0.1477684 (1200)	total: 1m 13s	remaining: 3m 52s
1500:	learn: 0.1318488	test: 0.1470102	best: 0.1470102 (1500)	total: 1m 32s	remaining: 3m 34s
1800:	learn: 0.1286874	test: 0.1465720	best: 0.1465686 (1798)	total: 1m 51s	remaining: 3m 17s
2100:	learn: 0.1256997	test: 0.1461930	best: 0.1461823 (2094)	total: 2m 9s	remaining: 2m 59s
2400:	learn: 0.1228935	test: 0.1459840	best: 0.1459637 (2387)	total: 2m 28s	remaining: 2m 40s
2700:	learn: 0.1202333	test: 0.1457086	best: 0.1457086 (2700)	total: 2m 47s	remaining: 2m 22s
3000:	learn: 0.1177097	test: 0.1456172	best: 0.1456172 (3000)	total: 3m 6s	rem

0:	learn: 0.5895435	test: 0.5895196	best: 0.5895196 (0)	total: 87ms	remaining: 7m 14s
300:	learn: 0.1515617	test: 0.1517786	best: 0.1517786 (300)	total: 18.7s	remaining: 4m 51s
600:	learn: 0.1444587	test: 0.1477323	best: 0.1477323 (600)	total: 37s	remaining: 4m 31s
900:	learn: 0.1396266	test: 0.1460394	best: 0.1460394 (900)	total: 55.6s	remaining: 4m 12s
1200:	learn: 0.1357130	test: 0.1452133	best: 0.1452018 (1198)	total: 1m 13s	remaining: 3m 53s
1500:	learn: 0.1321890	test: 0.1446577	best: 0.1446568 (1493)	total: 1m 32s	remaining: 3m 35s
1800:	learn: 0.1289173	test: 0.1441479	best: 0.1441479 (1800)	total: 1m 51s	remaining: 3m 17s
2100:	learn: 0.1259642	test: 0.1438723	best: 0.1438697 (2093)	total: 2m 9s	remaining: 2m 59s
2400:	learn: 0.1231733	test: 0.1436302	best: 0.1436293 (2399)	total: 2m 28s	remaining: 2m 40s
2700:	learn: 0.1205716	test: 0.1434350	best: 0.1434290 (2688)	total: 2m 47s	remaining: 2m 22s
3000:	learn: 0.1181319	test: 0.1433387	best: 0.1433351 (2996)	total: 3m 6s	remai

...

Epoch 1/20


I0000 00:00:1754787766.914388     296 service.cc:148] XLA service 0x7b052c010e90 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1754787766.915083     296 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1754787766.915122     296 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1754787767.585435     296 cuda_dnn.cc:529] Loaded cuDNN version 90300


 30/740 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - auc: 0.7245 - loss: 0.8267

I0000 00:00:1754787771.577623     296 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


740/740 ━━━━━━━━━━━━━━━━━━━━ 22s 14ms/step - auc: 0.8716 - loss: 0.3629 - val_auc: 0.8861 - val_loss: 0.3478 - learning_rate: 0.0100
Epoch 2/20
740/740 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - auc: 0.9155 - loss: 0.2762 - val_auc: 0.9003 - val_loss: 0.3170 - learning_rate: 0.0100
Epoch 3/20
740/740 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - auc: 0.9230 - loss: 0.2651 - val_auc: 0.9159 - val_loss: 0.3146 - learning_rate: 0.0100
Epoch 4/20
740/740 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - auc: 0.9283 - loss: 0.2570 - val_auc: 0.9244 - val_loss: 0.2703 - learning_rate: 0.0100
Epoch 5/20
740/740 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - auc: 0.9337 - loss: 0.2482 - val_auc: 0.9298 - val_loss: 0.2730 - learning_rate: 0.0100
Epoch 6/20
740/740 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - auc: 0.9397 - loss: 0.2378 - val_auc: 0.9416 - val_loss: 0.2410 - learning_rate: 1.0000e-03
Epoch 7/20
740/740 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - auc: 0.9427 - loss: 0.2321 - val_auc: 0.9454 - val_loss: 0.2380 - learning_rate: 1.0000e-03
Epoch 8

In [29]:
X = logit(pd.DataFrame(oof_pred_probs).clip(1e-15, 1-1e-15))
X_test = logit(pd.DataFrame(test_pred_probs).clip(1e-15, 1-1e-15))


In [30]:
from sklearn.metrics import roc_auc_score
from koolbox import Trainer

class CFG:

    
    n_folds = 7
    seed = 42
    
    cv = StratifiedKFold(n_splits=n_folds, random_state=seed, shuffle=True)
    metric = roc_auc_score
    
    n_optuna_trials = 200

In [31]:
def objective(trial):
    solver_penalty_options = [
        ('liblinear', 'l1'),
        ('liblinear', 'l2'),
        ('lbfgs', 'l2'),
        ('lbfgs', None),
        ('newton-cg', 'l2'),
        ('newton-cg', None),
        ('newton-cholesky', 'l2'),
        ('newton-cholesky', None)
    ]
    solver, penalty = trial.suggest_categorical('solver_penalty', solver_penalty_options)
    
    params = {
        'random_state': CFG.seed,
        'max_iter': 1000,
        'C': trial.suggest_float('C', 0, 1),
        'tol': trial.suggest_float('tol', 1e-6, 1e-2),
        'fit_intercept': trial.suggest_categorical('fit_intercept', [True, False]),
        'class_weight': trial.suggest_categorical('class_weight', ['balanced', None]),
        'solver': solver,
        'penalty': penalty
    }
    
    
    trainer = Trainer(
        LogisticRegression(**params),
        cv=CFG.cv,
        metric=CFG.metric,
        metric_precision=6,
        use_early_stopping=False,
        verbose=False,
        task="binary",
    )
    trainer.fit(pd.DataFrame(X), true)
    
    return np.mean(trainer.fold_scores)

sampler = optuna.samplers.TPESampler(seed=CFG.seed, multivariate=True, n_startup_trials=CFG.n_optuna_trials // 10)
study = optuna.create_study(direction='maximize', sampler=sampler)
study.optimize(objective, n_trials=CFG.n_optuna_trials, n_jobs=-1)
best_params = study.best_params


[I 2025-08-10 01:16:54,598] A new study created in memory with name: no-name-a40b098f-eedf-4608-b616-67e0dd737b17
[I 2025-08-10 01:17:28,814] Trial 0 finished with value: 0.9794462729322978 and parameters: {'solver_penalty': ('lbfgs', None), 'C': 0.7835325298385214, 'tol': 0.0017347904017452342, 'fit_intercept': True, 'class_weight': 'balanced'}. Best is trial 0 with value: 0.9794462729322978.
[I 2025-08-10 01:17:29,346] Trial 3 finished with value: 0.9776495914589093 and parameters: {'solver_penalty': ('lbfgs', 'l2'), 'C': 0.15516295392423796, 'tol': 0.0011449751401777244, 'fit_intercept': False, 'class_weight': 'balanced'}. Best is trial 0 with value: 0.9794462729322978.
[I 2025-08-10 01:17:35,437] Trial 1 finished with value: 0.9794965196410497 and parameters: {'solver_penalty': ('newton-cg', 'l2'), 'C': 0.3689855009247758, 'tol': 0.00861409203372812, 'fit_intercept': True, 'class_weight': 'balanced'}. Best is trial 1 with value: 0.9794965196410497.
[I 2025-08-10 01:17:37,720] Trial

In [32]:
solver, penalty = best_params['solver_penalty']
lr_params = {
    'random_state': CFG.seed,
    'max_iter': 1000,
    'C': best_params['C'],
    'tol': best_params['tol'],
    'fit_intercept': best_params['fit_intercept'],
    'class_weight': best_params['class_weight'],
    'solver': solver,
    'penalty': penalty
}

In [33]:
lr_trainer = Trainer(
    LogisticRegression(**lr_params),
    cv=CFG.cv,
    metric=CFG.metric,
    metric_precision=6,
    use_early_stopping=False,
    task="binary",
)

lr_trainer.fit(pd.DataFrame(X), true)

scores["LogisticRegression"] = lr_trainer.fold_scores
lr_test_pred_probs = lr_trainer.predict(X_test)
print(np.mean(scores["LogisticRegression"]))

Training LogisticRegression

--- Fold 0 - roc_auc_score: 0.979728 - Time: 0.89 s
--- Fold 1 - roc_auc_score: 0.979677 - Time: 0.89 s
--- Fold 2 - roc_auc_score: 0.978704 - Time: 0.90 s
--- Fold 3 - roc_auc_score: 0.979885 - Time: 0.89 s
--- Fold 4 - roc_auc_score: 0.979549 - Time: 0.89 s
--- Fold 5 - roc_auc_score: 0.979818 - Time: 0.89 s
--- Fold 6 - roc_auc_score: 0.979469 - Time: 0.81 s

------ Overall roc_auc_score: 0.979547 - Mean roc_auc_score: 0.979547 ± 0.000369 - Time: 9.29 s
0.9795470488130865


In [34]:
sub = pd.read_csv("/kaggle/input/playground-series-s5e8/sample_submission.csv")
sub['y']=lr_test_pred_probs

sub.to_csv("submission.csv",index=False)
sub.head()

,id,y
0,750000,0.001288
1,750001,0.082360
2,750002,0.000795
3,750003,0.000094
4,750004,0.016941
